In [1]:
%load_ext autoreload
%autoreload 2
%matplotlib notebook

import numpy as np
import matplotlib.pyplot as plt
from scipy.optimize import root
import h5py
import os
import itertools
from utils import *

## Playing with data

In [2]:
def read_cfg_Npt_explicit(folder, stem, N, n_t, cfg):
#     file = folder + '/' + stem + '.1460'
    file = folder + '/' + stem + '.' + str(cfg)
    if N == 2:
        Cnpt = np.zeros((n_t, n_t), dtype = np.complex64)
    else:
        Cnpt = np.zeros((24, n_t, n_t, n_t), dtype = np.complex64)
    with open(file) as f:
        lines = [line.rstrip() for line in f]
    for l in lines:
        x = l.split()
        if N == 2:
            tsrc, tsnk = int(x[0]), int(x[1])
            entry = np.complex(float(x[2]), float(x[3]))
            Cnpt[idx, tsrc, tsnk] = entry
        else:           # read 3-pt contraction structure
            tminus, tx, tplus = int(x[0]), int(x[1]), int(x[2])
            for i in range(24):
                re, im = float(x[2 * i + 3]), float(x[2 * i + 4])
                Cnpt[i, tminus, tx, tplus] = np.complex(re, im)
    return Cnpt

In [3]:
config = '24I/ml_0p01'
resultsA = '/Users/theoares/Pi_Pi/' + config + '/resultsA'
stem_3pt = 'pion_0vbb_4quark'
l, t = 24, 64
L = Lattice(l, t)

In [4]:
C3pt = read_cfg_Npt_explicit(resultsA, stem_3pt, 3, L.T, 1580)

In [ ]:
# x = np.zeros((L.T), dtype = np.complex64)
# for t in range(L.T):
#     x[t] = C3pt[0, t, 7, t]
# print(np.average(x))

In [ ]:
# x = []
# for t1 in range(L.T):
#     for t2 in range(t1, L.T):
#         if np.abs(t1 - t2) == 1:
#             x.append(C3pt[0, t1, 1, t2])    # should avg to first operator on line 1, 1

In [ ]:
# sum(x) / len(x)

In [ ]:
# C3pt_tavg = np.zeros((24, L.T, L.T), dtype = np.complex64)    # (tx, |t+ - t-|)
# # cnt = np.zeros((L.T, L.T))
# for tminus, tx, tplus in itertools.product(range(L.T), repeat = 3):
#     t1 = (tx - tminus) % L.T
#     t2 = (tplus - tminus) % L.T
#     if t1 == 1 and t2 == 2:
#         print((tminus, tx, tplus))
#     C3pt_tavg[:, t1, t2] += C3pt[:, tminus, tx, tplus]    # need to average about tminus = 0 first
# C3pt_tavg = C3pt_tavg / L.T

In [ ]:
# C3pt.shape[:-1]

In [5]:
C3pt_tavg = np.zeros((24, L.T, L.T), dtype = np.complex64)    # (tx, |t+ - t-|)
cnt = np.zeros((L.T, L.T))
for tminus, tx, tplus in itertools.product(range(L.T), repeat = 3):
    C3pt_tavg[:, tminus, tplus] += C3pt[:, tminus, tx, tplus]    # need to average about tminus = 0 first
    cnt[tminus, tplus] += 1
for tminus, tplus in itertools.product(range(L.T), repeat = 2):
    C3pt_tavg[:, tminus, tplus] = C3pt_tavg[:, tminus, tplus] / cnt[tminus, tplus]

In [6]:
C3pt_refl = np.zeros((24, L.T, L.T), dtype = np.complex64)
for t1, t2 in itertools.product(range(L.T), repeat = 2):
    C3pt_refl[:, t1, t2] = C3pt_tavg[:, (L.T - t1) % L.T, (L.T - t2) % L.T]

In [7]:
# first t index is in the file name, second is the label in the file
C_sep = np.zeros((24, L.T, L.T), dtype = np.complex64)
C_sep_refl = np.zeros((24, L.T, L.T), dtype = np.complex64)
for sep, t in itertools.product(range(L.T), repeat = 2):
    C_sep[:, sep, t] = C3pt_tavg[:, (L.T - sep) % L.T, (L.T - sep + t) % L.T]
    C_sep_refl[:, sep, t] = C3pt_refl[:, (L.T - sep) % L.T, (L.T - sep + t) % L.T]

In [ ]:
C_sep_refl[:, 35, 6]

Ex: Line 2 42 in pion_0vbb_4quark.1460 is 2.576e2 0 3.21e2 ~0, line 2 in pion_0vbb_4quark.42.1460 is 2.27e4 0 2.97e4 ~0, so these seem to be different things. I'd assume the average is done in a different way or something, but would like to figure out how to get each of these numbers from the raw data

Maybe try to fold over the x insertion and / or the midpoint of the lattice and see what we get?

_ref is the reflected file, see resultsA.tavg/run.sh.

The time averager is now giving me what David's files have, it seems to be averaged over tx.
Next: do reflections, and figure out the difference between pion_0vbb_4quark.1460 and pion_0vbb_4quark.1.1460 (and all other separations) in resultsA.tavg

So I found a hidden .awk file in one of the directories in the project that does the time averaging, and managed to use that to replicate the data in the time-averaged folder in my python script. It looks like the time averaging is actually over operator insertion times-- unless David mislabeled the time labels in his writeup for the 3 point function with (t-, tx, t+), the time averaged file at (t1, t2) is actually $(t_-, t_+)$ averaged over tx. I’m not sure why he says the labels in the time averaged files are $(t_x, |t_+ - t_-|)$, does this make any sense to you?

First notes: 
pion_0vbb_4quark.1460: Label 63, 63 matches up with pion_0vbb_4quark.1.1460 Label 0, except the imaginary parts are sometimes opposite. Same with the following:

pion_0vbb_4quark.SEP.CFG: Rows labeled by $\Delta t$ and the file is labeled by $t_{sep}$, where we're looking at the information for $t_- = T - t_{sep}$ and $t_+ = t_- + \Delta t$. 

Label in .1460 <--> Label in .1.1460
63, 63         <-->         0
63, 0          <-->         1
1, 2 and 63, 1 <-->         2

Label in .1460 <--> Label in .9.1460
55, 57         <-->         2

Label in .1460 <--> Label in .24.1460
40, 47         <-->         7

TODO: I don't need C_sep, it's not right. To get pion_0vbb_4quark.SEP.CFG at line t, just index C3pt_tavg[CFG, :, SEP, t] (where CFG just indexes the config number in the files)

In [ ]:
# now figure out the .12.1460 files, etc.

In [11]:
C3pt_tavg[:, 1, 8]

array([ 3.13151031e+05+0.0000000e+00j,  3.91712344e+05-9.7637776e-15j,
        1.83089188e+05-7.6384436e-16j,  2.63706625e+05+2.6705932e-16j,
        7.27447250e+05+0.0000000e+00j,  4.41469156e+05+7.5880532e-15j,
        3.13100781e+05+8.3280766e-16j,  2.83541344e+05-4.3183105e-16j,
       -8.93180566e+03-4.7593675e-16j, -1.23001734e+05+5.2565111e-15j,
       -5.67667695e+04-2.7303000e-15j, -4.25199000e+05+1.7879576e-15j,
        2.18995789e-16+1.5303595e+02j,  7.57056427e+01+3.6468773e+01j,
       -4.15108189e-15+1.8275929e+02j,  3.71212067e+02-4.9737854e+01j,
       -4.55564516e-15-5.3560299e+01j, -7.57056427e+01+3.6468773e+01j,
        1.70589820e-15-2.5569691e+02j, -3.71212067e+02-4.9737854e+01j,
        3.07376465e+04+1.8489937e-15j,  1.37021453e+05+3.4889795e-16j,
        4.27470742e+04-3.7518219e-17j,  4.03393250e+05+1.3814176e-16j],
      dtype=complex64)

In [12]:
# C3pt_tavg[:, 18, 7]

In [13]:
C_sep[:, 1, 8]

array([ 3.13151031e+05+0.0000000e+00j,  3.91712344e+05+9.7637776e-15j,
        1.83089219e+05+7.6384436e-16j,  2.63706625e+05-2.6705932e-16j,
        7.27447250e+05+0.0000000e+00j,  4.41469156e+05-7.5880532e-15j,
        3.13100781e+05-8.3280766e-16j,  2.83541375e+05+4.3183105e-16j,
       -8.93180469e+03-4.7593670e-16j, -1.23001734e+05-1.6767737e-16j,
       -5.67667656e+04-2.2600357e-15j, -4.25199031e+05-1.8889898e-15j,
       -4.55564516e-15-5.3560276e+01j, -7.57055817e+01+3.6468781e+01j,
       -1.36809861e-15-2.5569682e+02j, -3.71211853e+02-4.9737846e+01j,
        2.18995922e-16+1.5303595e+02j,  7.57055817e+01+3.6468781e+01j,
       -4.96224384e-15+1.8275925e+02j,  3.71211853e+02-4.9737846e+01j,
        3.07376465e+04+1.8489944e-15j,  1.37021453e+05-9.2718185e-16j,
        4.27470742e+04+4.5166973e-16j,  4.03393188e+05-1.3064698e-15j],
      dtype=complex64)